This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_uploads/pytorch-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: Pytorch Multiple Input Output

The following tutorial demonstrates how to upload a Pytorch Multiple Input Output model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a Pytorch Multiple Input Output to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* A Wallaroo version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:

wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline


import string
import random

# make a random 4 character suffix to prevent overwriting other user's workspaces
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix=''

workspace_name = f'pytorch-multi-io{suffix}'
pipeline_name = f'pytorch-multi-io'

model_name = 'pytorch-multi-io'
model_file_name = "./models/model-auto-conversion_pytorch_multi_io_model.pt"

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for PyTorch models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a PyTorch model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, PyTorch model Required*) | Set as the `Framework.PyTorch`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The input schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The output schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
| `convert_wait` | `bool` (*Upload Method Optional, PyTorch model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input_1', pa.list_(pa.float32(), list_size=10)),
    pa.field('input_2', pa.list_(pa.float32(), list_size=5))
])
output_schema = pa.schema([
    pa.field('output_1', pa.list_(pa.float32(), list_size=3)),
    pa.field('output_2', pa.list_(pa.float32(), list_size=2))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.PYTORCH`.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,pytorch-multi-io
Version,a8e42173-391f-446f-989d-bd8a4d13eb0d
File Name,model-auto-conversion_pytorch_multi_io_model.pt
SHA,792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8
Status,ready
Image Path,None
Architecture,None
Acceleration,None
Updated At,2024-13-Feb 19:09:20


In [7]:
model.config().runtime()

'onnx'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
# clear the pipeline if it was used before
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.100.1.152',
   'name': 'engine-7b8f694d5c-4r9jw',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pytorch-multi-io',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'pytorch-multi-io',
      'version': 'a8e42173-391f-446f-989d-bd8a4d13eb0d',
      'sha': '792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.100.0.90',
   'name': 'engine-lb-dcd9c8cd7-njj76',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [10]:
mock_inference_data = [np.random.rand(10, 10), np.random.rand(10, 5)]
mock_dataframe = pd.DataFrame(
    {
        "input_1": mock_inference_data[0].tolist(),
        "input_2": mock_inference_data[1].tolist(),
    }
)

In [11]:
pipeline.infer(mock_dataframe)

,time,in.input_1,in.input_2,out.output_1,out.output_2,anomaly.count
0,2024-02-13 19:09:38.950,"[0.6841677322, 0.1807222014, 0.4044378146, 0.2...","[0.4666296263, 0.9242240276, 0.7538949394, 0.6...","[-0.10101657, 0.058848124, 0.28901264]","[-0.12994565, -0.10862556]",0
1,2024-02-13 19:09:38.950,"[0.1661862341, 0.3172618066, 0.3830702442, 0.3...","[0.1306921405, 0.9781285472, 0.23512599, 0.584...","[-0.09766423, -0.012812179, 0.252462]","[-0.07967451, -0.1563291]",0
2,2024-02-13 19:09:38.950,"[0.1045097892, 0.3283348476, 0.4891823965, 0.3...","[0.9959155436, 0.5100609033, 0.9112522474, 0.7...","[-0.036320925, -0.1215433, 0.32613605]","[-0.06854919, 0.08576059]",0
3,2024-02-13 19:09:38.950,"[0.0711935278, 0.1454028233, 0.8773375856, 0.7...","[0.5942909393, 0.3638316714, 0.7758301051, 0.6...","[-0.032149002, -0.08917303, 0.28324488]","[0.150363, 0.05229862]",0
4,2024-02-13 19:09:38.950,"[0.5132806823, 0.2214367759, 0.1522990439, 0.8...","[0.1184583246, 0.2849093711, 0.4934625845, 0.6...","[-0.07428986, -0.0474118, 0.18959253]","[-0.021057557, -0.093709774]",0
5,2024-02-13 19:09:38.950,"[0.4624022797, 0.2883695717, 0.6251142116, 0.8...","[0.8308445333, 0.7138760827, 0.7440608071, 0.2...","[-0.025463045, -0.061832134, 0.16245914]","[0.024273798, 0.015264243]",0
6,2024-02-13 19:09:38.950,"[0.5190652023, 0.3741201072, 0.2891511983, 0.7...","[0.7821363998, 0.9289637841, 0.8958883474, 0.6...","[-0.078385934, -0.014036655, 0.21711908]","[-0.014037868, -0.12455477]",0
7,2024-02-13 19:09:38.950,"[0.4376977187, 0.5516232433, 0.3359479437, 0.7...","[0.5330120259, 0.3526138879, 0.4385310666, 0.6...","[-0.045134243, -0.0151004195, 0.19396316]","[0.018248288, 0.0025458485]",0
8,2024-02-13 19:09:38.950,"[0.420121827, 0.9279616795, 0.3745078043, 0.85...","[0.7552034807, 0.7273257552, 0.0195262221, 0.6...","[-0.058705688, -0.014976103, 0.1577933]","[-0.15129606, 0.028029606]",0
9,2024-02-13 19:09:38.950,"[0.2407957153, 0.1921562214, 0.1859864299, 0.4...","[0.7745044138, 0.6805588759, 0.4389954558, 0.2...","[-0.054730702, -0.055193268, 0.20420705]","[-0.12578416, 0.044233248]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [12]:
pipeline.undeploy()

name,pytorch-multi-io
created,2024-02-13 19:09:08.686761+00:00
last_updated,2024-02-13 19:09:22.224009+00:00
deployed,False
arch,None
accel,None
tags,
versions,"8e8c9d94-3bb8-4c70-9867-a207a786f8c2, 621d3e8f-ad9d-4612-a4c0-dd10ffe2b523"
steps,pytorch-multi-io
published,False
